In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from multiprocessing import Pool

from sklearn.neighbors import KDTree

In [13]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_5.zip'):
    if cnt == 4:
        break
    cnt += 1

In [16]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_5.zip'):
    if cnt == 5:
        break
    cnt += 1

In [3]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_sample.zip'):
    print(event_id)
    if cnt == 50:
        break
    cnt += 1

1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050


In [9]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm
from sklearn.cluster import DBSCAN

class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.528]):                        
        self.rz_scales=rz_scales
    
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 6
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        xm = np.mean(x,axis=0)
        x = x - xm
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    def _preprocess(self, hits):
        
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
    
    
    
    def find_labels(self, params):
        w1, w2, w3, w4, w5, w6, w7, epsilon = 2.7474448671796874,1.3649721713529086,0.7034918842926337,\
                                0.0005549122352940002,0.023096034747190672,0.04619756315527515,\
                                0.2437077420144654,0.009750302717746615
        
        hits, dz, z_shift, unroll_type = params

        hits['z'] = hits['z'] - z_shift
        hits['r'] = np.sqrt(hits['x'].values ** 2 + hits['y'].values ** 2 + hits['z'].values ** 2)
        hits['rt'] = np.sqrt(hits['x'].values ** 2 + hits['y'].values ** 2)
        hits['a0'] = np.arctan2(hits['y'].values, hits['x'].values)
        hits['z1'] = hits['z'].values / hits['rt'].values
        hits['z2'] = hits['z'].values / hits['r'].values
        hits['s1'] = hits['hit_id']
        hits['N1'] = 1
        hits['z1'] = hits['z'].values / hits['rt'].values
        hits['z2'] = hits['z'].values / hits['r'].values
        hits['x1'] = hits['x'].values / hits['y'].values
        hits['x2'] = hits['x'].values / hits['r'].values
        hits['x3'] = hits['y'].values / hits['r'].values
        hits['x4'] = hits['rt'].values / hits['r'].values
       
        #hits['a1'] = hits['a0'].values + np.nan_to_num(np.arccos(dz*hits['rt'].values))

        if unroll_type == 0:
            hits['a1'] = hits['a0'].values + np.nan_to_num(np.arccos(dz*hits['rt'].values))
        if unroll_type == 1:
            hits['a1'] = hits['a0'].values + dz*hits['rt'].values
        if unroll_type == 2:
            hits['a1'] = hits['a0'].values + dz*hits['z'].values
        if unroll_type == 3:
            hits['a1'] = hits['a0'].values + dz * (hits['rt'].values + 0.000005 * hits['rt'].values ** 2)
        #hits['a1'] = hits['a0'].values + np.nan_to_num(np.arccos(dz*hits['rt'].values))

        hits['sina1'] = np.sin(hits['a1'].values)
        hits['cosa1'] = np.cos(hits['a1'].values)
        ss = StandardScaler()
        hits = ss.fit_transform(hits[['sina1', 'cosa1', 'z1', 'z2','x1','x2','x3','x4']].values)
        cx = np.array([w1, w1, w2, w3, w4, w5, w6, w7])
        hits = np.multiply(hits, cx)
        clusters = DBSCAN(eps=0.009750302717746615, min_samples=1, metric="euclidean", n_jobs=32).fit(hits).labels_
        return clusters    
    
    
    
    
    def _init(self, hits, Niter):
        
#         w1, w2, w3, w4, w5, w6, w7, epsilon = 2.7474448671796874,1.3649721713529086,0.7034918842926337,\
#                                         0.0005549122352940002,0.023096034747190672,0.04619756315527515,\
#                                         0.2437077420144654,0.009750302717746615
        
        
        

        
        
        params = []
        for i in range(Niter):
            
            unroll_type = np.random.randint(0,4)
            if unroll_type == 0:
                dz = np.random.normal(0.0, 0.00035)
            elif unroll_type == 1:
                dz = np.random.normal(0.0, 0.00065)
            elif unroll_type == 2:
                dz = np.random.normal(0.0, 0.00085)
            elif unroll_type == 3:
                dz = np.random.normal(0.0, 0.001)

            
            
            
            #dz = 1 / 1000 * (ii / 2) / 180 * np.pi
            #dz = np.random.normal(0.0, 0.001)
            #dz = np.random.normal(0.0, 0.00035)
 
            z_shift = np.random.normal(0.0, 4.5)


            params.append((hits, dz, z_shift, unroll_type))
            
        pool = Pool(processes=8)
        result = []
        for i in tqdm(pool.imap(self.find_labels, params)):
            result += [i]
        pool.close()
                
            
        return np.array(result)
    
    def predict(self, hits, Niter):         
        result  = self._init(hits, Niter)
#         X = self._preprocess(hits) 
#         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                              metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
#         labels = np.unique(self.clusters)
#         self._eliminate_outliers(labels,X)          
#         max_len = np.max(self.clusters)
#         mask = self.clusters == 0
#         self.clusters[mask] = cl.fit_predict(X[mask])+max_len
        return result

In [7]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump([1, 2], f)

In [11]:
1 / 1000 * (338 / 2) / 180 * np.pi

0.002949606435870417

In [11]:


model = Clusterer()
result = model.predict(hits, 3000)



313it [00:59,  5.24it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
951it [02:55,  5.41it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
2238it [06:54,  5.40it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
3000it [09:16,  5.39it/s]


In [14]:


model = Clusterer()
result = model.predict(hits, 3000)


1671it [06:29,  4.29it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
2076it [07:58,  4.34it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
3000it [11:22,  4.39it/s]


In [17]:


model = Clusterer()
result = model.predict(hits, 3000)

665it [02:21,  4.70it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
865it [03:01,  4.76it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
2409it [08:09,  4.92it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
2509it [08:29,  4.92it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in arccos
3000it [10:10,  4.91it/s]


In [6]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [7]:
def merge(cl1, cl2, min_cnt): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) &  (d['N2'].values>min_cnt))
    #cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) )

    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 
    return s1

In [84]:
result = np.vstack([result, result0])

In [78]:
result0 =result.copy()

In [ ]:
scipy.sparse.linalg.svds

In [12]:
np.save('predicts/luis_3', result)

In [15]:
np.save('predicts/luis_4', result)

In [18]:
np.save('predicts/luis_5', result)

In [8]:
#result = np.array(second)
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result[:])):
        labels = merge(labels, result[i], k)

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.5922888474688374


In [ ]:
0.6103758646010513
0.5986615684933159
0.597
0.44631528825442496

In [26]:
submission = create_one_event_submission(0, hits['hit_id'].values, labels)
score = score_event(truth, submission)
print(score)


0.5071140015567802


In [ ]:
0.5071140015567802
0.5022115428740757

In [10]:
np.save('predicts/luis_all_1000_event', result)

In [13]:
result.shape

(3000, 125504)